In [0]:
#Tujuan: Mencari kolom yang akan dipertahankan
#Jawaban: chdrnum

In [0]:
# OPEN CHANDRA METADATA

from pyspark.sql import functions as F, types as T
from functools import reduce

# ===== CONFIG =====
CATALOG = "lapse_scoring_dev"
SCHEMA  = "01_bronze"

# pilih salah satu:
TABLES = [
    "chandra_metadata_bronze",
    # "chandra_201702_trad_master_bronze",
    # "pak_tani_golden_customers_360",
]
# atau pakai pola:
TABLE_PATTERN = None  # mis. "chandra_%_trad_master_bronze" (None = tidak dipakai)

def list_tables(catalog, schema, pattern=None):
    if pattern:
        q = f"""
        SELECT table_name
        FROM {catalog}.information_schema.tables
        WHERE table_schema = '{schema}'
          AND table_type = 'BASE TABLE'
          AND table_name LIKE '{pattern}'
        """
        return [r["table_name"] for r in spark.sql(q).collect()]
    return TABLES

def explore_table(fullname):
    print(f"\n=== EDA: {fullname} ===")
    df = spark.table(fullname)

    # ---- ringkasan cepat
    rows = df.count()
    print("Rows:", rows)
    display(df.limit(100))
    display(spark.createDataFrame(df.dtypes, ["column","type"]))
    print("\n=== END EDA ===")

tables = list_tables(CATALOG, "01_bronze", pattern=TABLE_PATTERN) if TABLE_PATTERN else TABLES
for t in tables:
    explore_table(f"{CATALOG}.`{SCHEMA}`.{t}")

#keep chdrnum, shortdesc, aracde, cnttype, crtable, hissdte, subdat, payfreq, sumins, statcode, lifenum, age02, dob02, gender, smokeind, annprem01, annprem02, ptdate, btdate, clntnum, age01, dob01, pldesc01, pldesc02, prat, orig_sum

In [0]:
# =========================
# EDA Presence Check (Bronze) + Row Count
# =========================
from pyspark.sql import functions as F

# ===== CONFIG =====
CATALOG = "lapse_scoring_dev"
SCHEMA  = "01_bronze"

# Daftar tabel (atau pakai TABLE_PATTERN)
TABLES = [
    "chandra_201701_trad_master_bronze",
    "chandra_201701_ul_master_bronze",
    "chandra_201702_trad_master_bronze",
    "chandra_201702_ul_master_bronze",
    "chandra_201703_trad_master_bronze",
    "chandra_201703_ul_master_bronze",
    "chandra_201704_trad_master_bronze",
    "chandra_201704_ul_master_bronze",
    "chandra_201705_trad_master_bronze",
    "chandra_201705_ul_master_bronze",
    "chandra_201706_trad_master_bronze",
    "chandra_201706_ul_master_bronze",
    "chandra_201707_trad_master_bronze",
    "chandra_201707_ul_master_bronze",
    "chandra_201708_trad_master_bronze",
    "chandra_201708_ul_master_bronze",
    "chandra_201709_trad_master_bronze",
    "chandra_201709_ul_master_bronze",
    "chandra_201710_trad_master_bronze",
    "chandra_201710_ul_master_bronze",
    "chandra_201711_trad_master_bronze",
    "chandra_201711_ul_master_bronze",
    "chandra_201712_trad_master_bronze",
    "chandra_201712_ul_master_bronze",
    "chandra_201801_trad_master_bronze",
    "chandra_201801_ul_master_bronze",
    "chandra_201802_trad_master_bronze",
    "chandra_201802_ul_master_bronze",
    "chandra_201803_trad_master_bronze",
    "chandra_201803_ul_master_bronze",
    "chandra_201804_trad_master_bronze",
    "chandra_201804_ul_master_bronze",
    "chandra_201805_trad_master_bronze",
    "chandra_201805_ul_master_bronze",
    "chandra_201806_trad_master_bronze",
    "chandra_201806_ul_master_bronze",
    "chandra_201807_trad_master_bronze",
    "chandra_201807_ul_master_bronze",
    "chandra_201808_trad_master_bronze",
    "chandra_201808_ul_master_bronze",
    "chandra_201809_trad_master_bronze",
    "chandra_201809_ul_master_bronze",
    "chandra_201810_trad_master_bronze",
    "chandra_201810_ul_master_bronze",
    "chandra_201811_trad_master_bronze",
    "chandra_201811_ul_master_bronze",
    "chandra_201812_trad_master_bronze",
    "chandra_201812_ul_master_bronze",
    "chandra_201901_trad_master_bronze",
    "chandra_201901_ul_master_bronze",
    "chandra_201902_trad_master_bronze",
    "chandra_201902_ul_master_bronze",
    "chandra_201903_trad_master_bronze",
    "chandra_201903_ul_master_bronze",
    "chandra_201904_trad_master_bronze",
    "chandra_201904_ul_master_bronze",
    "chandra_201905_trad_master_bronze",
    "chandra_201905_ul_master_bronze",
    "chandra_201906_trad_master_bronze",
    "chandra_201906_ul_master_bronze",
    "chandra_201907_trad_master_bronze",
    "chandra_201907_ul_master_bronze",
    "chandra_201908_trad_master_bronze",
    "chandra_201908_ul_master_bronze",
    "chandra_201909_trad_master_bronze",
    "chandra_201909_ul_master_bronze",
    "chandra_201910_trad_master_bronze",
    "chandra_201910_ul_master_bronze",
    "chandra_201911_trad_master_bronze",
    "chandra_201911_ul_master_bronze",
    "chandra_201912_trad_master_bronze",
    "chandra_201912_ul_master_bronze",
]
TABLE_PATTERN = None  # contoh: "chandra_%_trad_master_bronze"

# Kolom yang ingin dipastikan ada/seragam
WANTED = [
    "chdrnum","shortdesc","aracde","cnttype","crtable","hissdte","subdat",
    "payfreq","sumins","statcode","lifenum","age02","dob02","gender",
    "smokeind","annprem01","ptdate","btdate","clntnum",
    "age01","dob01","pldesc01","pldesc02"
]

# ===== Utils =====
def list_tables(catalog, schema, pattern=None):
    if pattern:
        q = f"""
        SELECT table_name
        FROM {catalog}.information_schema.tables
        WHERE table_schema = '{schema}'
          AND table_type = 'BASE TABLE'
          AND table_name LIKE '{pattern}'
        """
        return [r["table_name"] for r in spark.sql(q).collect()]
    return TABLES

def build_presence_report(catalog, schema, tables, wanted):
    rows = []
    wanted_set = set(wanted)
    for t in tables:
        fullname = f"{catalog}.`{schema}`.{t}"
        df = spark.table(fullname)
        cols = set(df.columns)
        missing = sorted(list(wanted_set - cols))
        extra   = sorted(list(cols - wanted_set))
        row_cnt = df.count()  # row count

        rows.append({
            "table_name": t,
            "row_count": row_cnt,
            "wanted_count": len(wanted),
            "present_count": len(wanted_set & cols),
            "missing_count": len(missing),
            "missing_cols": ", ".join(missing) if missing else "",
            "extra_count": len(extra),
            "extra_sample": ", ".join(extra[:10])
        })
    return spark.createDataFrame(rows)

def build_column_matrix(catalog, schema, tables, wanted):
    rows = []
    for t in tables:
        fullname = f"{catalog}.`{schema}`.{t}"
        cols = set(spark.table(fullname).columns)
        for w in wanted:
            rows.append({"table_name": t, "column": w, "present": int(w in cols)})
    df = spark.createDataFrame(rows)
    return (df.groupBy("table_name")
              .pivot("column", wanted)
              .agg(F.max("present"))
              .orderBy("table_name"))

def select_with_missing_as_null(df, wanted, default_type="string"):
    cols = df.columns
    selected = []
    for c in wanted:
        if c in cols:
            selected.append(F.col(c))
        else:
            selected.append(F.lit(None).cast(default_type).alias(c))
    return df.select(*selected)  # urutan sesuai WANTED

def explore_table_selected(fullname: str):
    print(f"\n=== EDA (selected cols): {fullname} ===")
    df = spark.table(fullname)

    existing = [c for c in WANTED if c in df.columns]
    missing  = [c for c in WANTED if c not in df.columns]

    if missing:
        print(f"Missing columns ({len(missing)}): {missing}")
    else:
        print("All WANTED columns are present ✅")

    # versi NULL-fill agar konsisten & siap union
    df_sel = select_with_missing_as_null(df, WANTED)

    # tampilkan contoh & schema
    display(df_sel.limit(100))
    display(spark.createDataFrame(df_sel.dtypes, ["column","type"]))
    print("=== END EDA ===")

# ===== Run presence checks =====
tables = list_tables(CATALOG, SCHEMA, pattern=TABLE_PATTERN) if TABLE_PATTERN else TABLES

presence = build_presence_report(CATALOG, SCHEMA, tables, WANTED)
display(presence.orderBy(F.desc("missing_count")))

matrix = build_column_matrix(CATALOG, SCHEMA, tables, WANTED)

# Tambahkan row_count ke matrix (kolom pertama)
counts_df = spark.createDataFrame(
    [(t, spark.table(f"{CATALOG}.`{SCHEMA}`.{t}").count()) for t in tables],
    ["table_name", "row_count"]
)
matrix_with_counts = (matrix.join(counts_df, on="table_name", how="left")
                             .select("table_name","row_count", *[c for c in matrix.columns if c != "table_name"])
                             .orderBy("table_name"))

display(matrix_with_counts)

# (Opsional) simpan laporan ke Silver untuk dokumentasi/audit
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.`02_silver`")
presence.write.mode("overwrite").saveAsTable(f"{CATALOG}.`02_silver`.bronze_wanted_presence")
matrix_with_counts.write.mode("overwrite").saveAsTable(f"{CATALOG}.`02_silver`.bronze_wanted_matrix")

# ===== Preview per tabel =====
for t in tables:
    explore_table_selected(f"{CATALOG}.`{SCHEMA}`.{t}")


In [0]:
# ===== CONFIG =====
CATALOG = "lapse_scoring_dev"
SCHEMA  = "01_bronze"

# pilih salah satu:
TABLES = [
    "chandra_201701_trad_master_bronze",
    # "chandra_201702_trad_master_bronze",
    # "pak_tani_golden_customers_360",
]
# atau pakai pola:
TABLE_PATTERN = None  # mis. "chandra_%_trad_master_bronze" (None = tidak dipakai)


In [0]:
from pyspark.sql import functions as F, types as T
from functools import reduce

def list_tables(catalog, schema, pattern=None):
    if pattern:
        q = f"""
        SELECT table_name
        FROM {catalog}.information_schema.tables
        WHERE table_schema = '{schema}'
          AND table_type = 'BASE TABLE'
          AND table_name LIKE '{pattern}'
        """
        return [r["table_name"] for r in spark.sql(q).collect()]
    return TABLES

def is_num(dt):   return dt.startswith("int") or dt.startswith("bigint") or dt.startswith("double") or dt.startswith("float") or dt.startswith("decimal")
def is_date(dt):  return dt in ("date","timestamp")
def is_str(dt):   return dt.startswith("string") or dt.startswith("varchar") or dt.startswith("char")

def topk(df, col, k=10):
    return (df.groupBy(col).count().orderBy(F.desc("count")).limit(k))

def profile_numeric(df, col):
    return (df.select(
        F.count(F.col(col)).alias("non_null"),
        F.count(F.when(F.col(col).isNull(), col)).alias("nulls"),
        F.approx_count_distinct(col).alias("distinct_approx"),
        F.mean(col).alias("mean"),
        F.stddev(col).alias("stddev"),
        F.min(col).alias("min"),
        F.max(col).alias("max"),
    ).withColumn("column", F.lit(col)).select("column","non_null","nulls","distinct_approx","mean","stddev","min","max"))

def profile_date(df, col):
    return (df.select(
        F.count(F.col(col)).alias("non_null"),
        F.count(F.when(F.col(col).isNull(), col)).alias("nulls"),
        F.approx_count_distinct(col).alias("distinct_approx"),
        F.min(col).alias("min_date"),
        F.max(col).alias("max_date"),
    ).withColumn("column", F.lit(col)).select("column","non_null","nulls","distinct_approx","min_date","max_date"))

def profile_string(df, col):
    return (df.select(
        F.count(F.col(col)).alias("non_null"),
        F.count(F.when(F.col(col).isNull(), col)).alias("nulls"),
        F.approx_count_distinct(col).alias("distinct_approx"),
        F.min(F.length(F.col(col))).alias("min_len"),
        F.max(F.length(F.col(col))).alias("max_len"),
    ).withColumn("column", F.lit(col)).select("column","non_null","nulls","distinct_approx","min_len","max_len"))


In [0]:
def explore_table(fullname):
    print(f"\n=== EDA: {fullname} ===")
    df = spark.table(fullname)

    # ---- ringkasan cepat
    rows = df.count()
    print("Rows:", rows)
    display(df.limit(10))
    display(spark.createDataFrame(df.dtypes, ["column","type"]))

    # ---- profil kolom per tipe
    num_cols  = [c for c,t in df.dtypes if is_num(t)]
    date_cols = [c for c,t in df.dtypes if is_date(t)]
    str_cols  = [c for c,t in df.dtypes if is_str(t)]

    prof_parts = []

    for c in num_cols:
        prof_parts.append(profile_numeric(df, c))
    for c in date_cols:
        prof_parts.append(profile_date(df, c))
    for c in str_cols:
        prof_parts.append(profile_string(df, c))

    if prof_parts:
        prof = reduce(lambda a,b: a.unionByName(b, allowMissingColumns=True), prof_parts)
        display(prof)

    # ---- frekuensi top untuk kolom kategorikal penting
    for c in ["statcode","payfreq","gender","cnttype","cntcurr","pldesc01"]:
        if c in df.columns:
            print(f"\nTop values: {c}")
            display(topk(df, c, 10))

    # ---- cek duplikasi kunci (contoh CHDRNUM)
    if "chdrnum" in df.columns:
        dup = (df.groupBy("chdrnum").count().where("count > 1").orderBy(F.desc("count")).limit(20))
        print("\nPotential duplicates on chdrnum:")
        display(dup)

    # ---- validasi kode status & payfreq
    if "statcode" in df.columns:
        valid_stat = ["IF","PP","LA","SU","RD","SP","TR","DR","DH","EX","FL"]
        anomalies = df.where(~F.upper(F.col("statcode")).isin(valid_stat))
        print("\nSTATCODE anomalies (unknown):")
        display(anomalies.select("statcode").groupBy("statcode").count().orderBy(F.desc("count")).limit(50))

    if "payfreq" in df.columns:
        valid_pf = ["M","MONTHLY","Q","QUARTERLY","SA","SEMIANNUAL","H","HALFYEAR","A","ANNUAL","YEARLY"]
        bad_pf = df.where(~F.upper(F.col("payfreq")).isin(valid_pf))
        print("\nPAYFREQ anomalies (unknown):")
        display(bad_pf.select("payfreq").groupBy("payfreq").count().orderBy(F.desc("count")).limit(50))

    # ---- angka negatif/aneh (premi/sum insured/cash value)
    for c in ["insprm","annprem01","sumins","mscv","orig_sum"]:
        if c in df.columns:
            print(f"\nNegative or zero check: {c}")
            display(df.where(F.col(c) <= 0).select(c).groupBy(c).count().orderBy(c).limit(20))

    # ---- tanggal anomali (urutan & range)
    # contoh: ptdate (paid-to) tidak boleh > today + 7 hari; btdate >= hissdte
    today = F.current_date()
    if "ptdate" in df.columns:
        print("\nPTDATE future > 7 days:")
        display(df.where(F.col("ptdate") > F.date_add(today, 7)).select("chdrnum","ptdate").limit(50))
    if "btdate" in df.columns and "hissdte" in df.columns:
        print("\nBTDATE < HISSDTE (billing before issue?)")
        display(df.where(F.col("btdate") < F.col("hissdte")).select("chdrnum","hissdte","btdate").limit(50))

    # ---- whitespace/karakter aneh di ID
    for c in ["chdrnum","clntnum"]:
        if c in df.columns:
            print(f"\nWhitespace/odd chars in {c}:")
            display(df.where((F.col(c).rlike(r"^\s")) | (F.col(c).rlike(r"\s$")) | (F.col(c).rlike(r"[^A-Za-z0-9_-]")))
                      .select(c).groupBy(c).count().orderBy(F.desc("count")).limit(50))

    print("\n=== END EDA ===")


In [0]:
tables = list_tables(CATALOG, "01_bronze", pattern=TABLE_PATTERN) if TABLE_PATTERN else TABLES
for t in tables:
    explore_table(f"{CATALOG}.`{SCHEMA}`.{t}")
